In [ ]:
# This is jupyter version of the code
%pip install pyvis --user
%pip install treform --user

In [ ]:
from treform.topic_model.pyTextMinerTopicModel import pyTextMinerTopicModel
import treform as ptm
import tomotopy as tp
from tqdm import tqdm
#import Komoran
from treform.tokenizer import Komoran
from ExampleManager import PathManager

In [ ]:
#corpus = ptm.CorpusFromFieldDelimitedFileWithYear(PathManager.get('../sample_data/sample_dmr_input.txt'),doc_index=2,year_index=1)

corpus = ptm.CorpusFromFieldDelimitedFileWithYear('file.csv', doc_index= 1, year_index=0, delimiter=',')
pair_map = corpus.pair_map

In [ ]:
len(corpus.docs)

In [ ]:
pipeline = ptm.Pipeline(ptm.splitter.NLTK(),
                        Komoran(), # MeCab, Komoran, Okt, etc...
                        # Mecab requires manual installation! try eunjeon if you can
                        ptm.helper.POSFilter('NN*'), # Noun
                        ptm.helper.SelectWordOnly(),
                        ptm.helper.StopwordFilter(file=PathManager.get('../stopwords/stopwordsKor.txt'))
                        )

In [ ]:
result = pipeline.processCorpus(tqdm(corpus.docs))
text_data = []
for doc in result:
    new_doc = []
    for sent in doc:
        for _str in sent:
            if len(_str) > 0:
                new_doc.append(_str)
    text_data.append(new_doc)

In [ ]:
for doc in tqdm(result):
    new_doc = []
    for sent in doc:
        for _str in sent:
            if len(_str) > 0:
                new_doc.append(_str)
    text_data.append(new_doc)

In [ ]:
text_data[0] # example of text data

In [ ]:
# get memory size of text data
import sys
print(sys.getsizeof(text_data))
# pickle if smaller than 1GB
if sys.getsizeof(text_data) < 1<<30:
    import pickle
    with open('text_data.pickle', 'wb') as f:
        pickle.dump(text_data, f)

In [ ]:
topic_model = pyTextMinerTopicModel()
topic_number = 10
#dominant_topic_number = 6
#if dominant_topic_number >= topic_number:
#    dominant_topic_number = topic_number - 1

mdl=None
#mode is either lda, dmr, hdp, infer, ct, visualize, etc

mode='lda'
label = ''
if mode == 'lda':
    #LDA
    print('Running LDA')
    label='LDA'
    lda_model_name = './test.lda.bin'
    mdl=topic_model.lda_model(text_data, lda_model_name, topic_number)
    print('perplexity score ' + str(mdl.perplexity))

elif mode == 'dmr':
    #DMR
    print('Running DMR')
    label='DMR'
    dmr_model_name='./test.dmr.bin'
    mdl=topic_model.dmr_model(text_data, pair_map, dmr_model_name, topic_number)
    print('perplexity score ' + str(mdl.perplexity))

elif mode == 'hdp':
    print('Running HDP')
    label='HDP'
    hdp_model_name='./test.hdp.bin'
    mdl, topic_num=topic_model.hdp_model(text_data, hdp_model_name)
    topic_number=topic_num
    print('perplexity score ' + str(mdl.perplexity))

elif mode == 'hlda':
    print('Running HLDA')
    label='HLDA'
    hlda_model_name = './test.hlda.bin'
    mdl=topic_model.hlda_model(text_data, hlda_model_name)
    print('perplexity score ' + str(mdl.perplexity))

elif mode == 'ct':
    print('Running CT')
    label = 'CT'
    ct_model_name = './test.ct.bin'
    save_file = 'D:/python_workspace/treform/topic_network.html'
    mdl = topic_model.ct_model(text_data, ct_model_name, topic_number=topic_number, topic_network_result=save_file)

elif mode == 'infer':
    lda_model_name = './test.lda.bin'
    unseen_text='아사이 베리 블루베리 비슷하다'
    topic_model.inferLDATopicModel(lda_model_name, unseen_text)

else:
    print('No mode is selected')

In [ ]:
mode = 'lda'
model_name = f'./test.{mode}.bin'
topic_number = 10
if locals().get('topic_model') is None:
    topic_model = pyTextMinerTopicModel()

mdl = tp.LDAModel.load(model_name)
print("Model loaded")
mdl.load(model_name)
# The below code extracts this dominant topic for each sentence
# and shows the weight of the topic and the keywords in a nicely formatted output.
df_topic_sents_keywords, matrix = pyTextMinerTopicModel().format_topics_sentences(topic_number=topic_number, mdl=mdl)

In [ ]:
import sys
sys.getsizeof(df_topic_sents_keywords)
if sys.getsizeof(df_topic_sents_keywords) < 1<<30:
    import pickle
    with open('df_topic_sents_keywords.pickle', 'wb') as f:
        pickle.dump(df_topic_sents_keywords, f)
else:
    print('df_topic_sents_keywords is too big to dump')
# Now dump matrix if it is smaller than 1GB
if sys.getsizeof(matrix) < 1<<30:
    import pickle
    with open('matrix.pickle', 'wb') as f:
        pickle.dump(matrix, f)
else:
    print('matrix is too big to dump')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib

def tSNE(mdl, matrix, label, topic_number=10):
    from bokeh.plotting import figure, output_file, show
    from bokeh.models import Label
    from bokeh.io import output_notebook
    import matplotlib.colors as mcolors
    from sklearn.manifold import TSNE

    # Array of topic weights
    arr = pd.DataFrame(matrix).fillna(0).values

    # Dominant topic number in each doc
    topic_num = np.argmax(arr, axis=1)

    # tSNE Dimension Reduction
    tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
    tsne_lda = tsne_model.fit_transform(arr)

    n_topics = topic_number
    mycolors = np.array([color for name, color in matplotlib.colors.cnames.items()])
    plot = figure(title="t-SNE Clustering of {} " + label + "Topics".format(n_topics),
                  width=900, height=700)

    plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])

    show(plot)
topic_model.tSNE = tSNE # monkey patching

In [ ]:
def visualize_topic_model(topic_model, mdl, matrix, label=''):
    import pickle
    with open('df_topic_sents_keywords.pickle', 'rb') as f:
        df_topic_sents_keywords = pickle.load(f)
    # Format
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
    df_dominant_topic.head(10)

    # Sometimes we want to get samples of sentences that most represent a given topic.
    # This code gets the most exemplar sentence for each topic.
    dist_result_file_ = f'./dist_doc_word_count{label}.png'
    topic_model.distribution_document_word_count(df_topic_sents_keywords, df_dominant_topic, result_file=dist_result_file_)

    #When working with a large number of documents,
    # we want to know how big the documents are as a whole and by topic.
    #Let’s plot the document word counts distribution.
    dominant_result_file_ = f'./dominant_topic_word_count{label}.png'
    dominant_topic_number = 7
    topic_model.distribution_word_count_by_dominant_topic(df_dominant_topic,dominant_topic_number=dominant_topic_number, result_file=dominant_result_file_)

    # Though we’ve already seen what are the topic keywords in each topic,
    # a word cloud with the size of the words proportional to the weight is a pleasant sight.
    # The coloring of the topics I’ve taken here is followed in the subsequent plots as well.
    topic_cloud_result_file = f'./topic_word_cloud{label}.png'
    topic_number = mdl.k
    topic_model.word_cloud_by_topic(mdl, topic_number=topic_number,topic_cloud_result_file=topic_cloud_result_file)

    topic_keyword_result_file = f'./topic_keyword{label}.png'
    # Let’s plot the word counts and the weights of each keyword in the same chart.
    topic_model.word_count_by_keywords(mdl,matrix,topic_keyword_result_file=topic_keyword_result_file, topic_number=topic_number)

    topics_per_document = f'./topic_per_document{label}.png'
    topic_model.topics_per_document(mdl, start=0, end=10, topics_per_document=topics_per_document, topic_number=topic_number)

    #visualize documents by tSNE
    topic_model.tSNE(mdl,matrix,'',topic_number=topic_number)

    visualization_file=f'./topic_visualization{label}.html'
    topic_model.make_pyLDAVis(mdl,visualization_file=visualization_file)

In [ ]:
def visualize(mode, topic_number, topic_model=None, return_matrix=False):
    model_name = f'./test.{mode}.bin'
    topic_number = 10
    if locals().get('topic_model') is None:
        topic_model = pyTextMinerTopicModel()
    if model_name == './test.lda.bin':
        mdl = tp.LDAModel.load(model_name)
        print("Model loaded")
    elif model_name == './test.dmr.bin':
        mdl = tp.DMRModel.load(model_name)
        visual_result_file1= './dmr_line_graph.png'
        visual_result_file2 = './dmr_bar_graph.png'
        pyTextMinerTopicModel().visualizeDMR(mdl,visual_result1=visual_result_file1, visual_result2=visual_result_file2)
    elif model_name == './test.ct.bin':
        mdl = tp.CTModel.load(model_name)
        result_file = './topic_network.html'
        topic_model.visualize_ct_model(mdl, topic_network_result=result_file)
    else:
        raise Exception("Cannot visualize this model {}".format(model_name))
    if return_matrix:
        df_topic_sents_keywords, matrix = pyTextMinerTopicModel().format_topics_sentences(topic_number=topic_number, mdl=mdl)
    else:
        matrix = None
    return mdl, matrix


In [ ]:
# now dmr

# load from 'text_data.pickle'
import pickle
with open('text_data.pickle', 'rb') as f:
    text_data = pickle.load(f)
corpus = ptm.CorpusFromFieldDelimitedFileWithYear('3_class_naver_news.csv', doc_index=4, year_index=0, delimiter=',')
pair_map = corpus.pair_map
mode = 'dmr'
print('Running DMR')
label='DMR'
dmr_model_name='./test.dmr.bin'
mdl=pyTextMinerTopicModel().dmr_model(text_data, pair_map, dmr_model_name, 10)
print('perplexity score ' + str(mdl.perplexity))

In [ ]:
# now visualize
model, matrix = visualize(mode, 10, mdl)

In [ ]:
# if matrix does not exist, pickle
import pickle
import os
import sys
if not os.path.exists('matrix.pickle'):
    print('matrix does not exist, pickle')
    if sys.getsizeof(matrix) > 100000000:
        print('matrix is too big, skip')
    else:
        with open('matrix.pickle', 'wb') as f:
            pickle.dump(matrix, f)

visualize_topic_model(pyTextMinerTopicModel(), mdl, matrix, label='DMR')